<a href="https://colab.research.google.com/github/Mohit501/ACGANS/blob/main/ACGANs_on_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing required libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Dense,Conv2D,Reshape,Flatten,Conv2DTranspose,Input,BatchNormalization,LeakyReLU,concatenate,Activation
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
import math

## Building a Generator

In [ ]:
def Generator(inputs,image_size,activation = 'sigmoid',labels = None,codes = None):
    image_resize = image_size//4
    kernel_size = 5
    layer_filters = [128.64,32,1]
    if labels is not None:
       if codes is None:
          inputs = [inputs,labels]
       else:
         inputs = [inputs,labels]+codes
       x = concatenate(inputs,axis=1)
    elif codes is not None:
         inputs = [inputs,code]
         x = concatenate(inputs,axis = 1)
    else:
         x = inputs
    x = Dense(image_resize*image_resize*128)(x)
    x = Reshape((image_resize,image_resize,128))(x)
    for filter in layer_filters:
        if filter > layer_filters[2]:
           strides = 2
        else:
           strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters = filter,kernel_size=kernel_size,strides=strides,padding='same')(x)
    if activation is not None:
       x = Activation('sigmoid')(x)
    generator = Model(inputs,x,name = 'generator')
    return generator

## Building a Discriminator

In [ ]:
def Discriminator(inputs,activation = 'sigmoid',num_labels = None,num_codes = None):
    kernel_size = 5
    layer_filters = [32,64,128,256]
    x = inputs
    for filter in layer_filters:
        if filter == layer_filters[-1]:
          strides = 1
        else: 
          strides = 2
        x = LeakyReLU(alpha = 0.2)(x)
        x = Conv2D(filters = filter,strides=strides,kernel_size=kernel_size,padding='same')(x)
    x = Flatten()(x)
    outputs = Dense(1)(x)
    if activation  is not None:
       print(activation)
       outputs = Activation(activation)(outputs)
    if num_labels:
      layer = Dense(layer_filters[-2])(x)
      labels = Dense(num_labels)(layer)
      labels= Activation('softmax',name = 'label')(labels)
      if num_codes is None:
        outputs = [outputs,labels]
      else:
        code1 = Dense(1)(layer)
        code1 = Activation('sigmoid')(code1)
        code2 = Dense(1)(layer)
        code2 = Activation('sigmoid')(code2)
        outputs = [outputs,labels,code1,code2]
    elif num_codes is not None:
        z0_recon = Dense(num_codes)(x)
        z0_recon = Activation('')(z0_recon)
        outputs = [outputs,z0_recon]
    discriminator = Model(inputs,outputs,name = 'Discriminator')
    return discriminator

# Building a Train Function

In [ ]:
def train(models,data,params):
    generator,discriminator,adversarial = models
    x_train,y_train = data
    batch_size, latent_size, train_steps, num_labels, model_name = params
    save_interval = 500
    noise_input = np.random.uniform(-1.0,1.0,size = [16,latent_size])
    noise_label = np.eye(num_labels)[np.arange(0,16)% num_labels]
    train_size = x_train.shape[0]
    print(model_name,'Labels for generated images:',np.argmax(noise_label,axis=1))
    for i in range (train_steps):
        rand_indexes = np.random.randint(0,train_size,size = batch_size)
        real_images = x_train[rand_indexes]
        real_labels = y_train[rand_indexes]
        noise = np.random.uniform(-1.0,1.0,size=[batch_size,latent_size])
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]
        fake_images = generator.predict([noise,fake_labels])
        x = np.concatenate((real_images,fake_images))
        labels = np.concatenate((real_labels,fake_labels))
        y = np.ones([2*batch_size,1])
        y[batch_size:,:] = 0 
        metrics = discriminator.train_on_batch(x,[y,labels])
        fmt = "%d [discriminator loss: %f , src loss: %f, lblloss: %f, srcacc: %f, lbl acc: %f]"
        log = fmt % (i,metrics[0],metrics[1],metrics[2],metrics[3],metrics[4])
       
        noise = np.random.uniform(-1.0,1.0,size=[batch_size,latent_size])
        fake_labels = np.eye(num_labels)[np.random.choice(num_labels,batch_size)]
        y = np.ones([batch_size,1])
        metrics = adversarial.train_on_batch([noise,fake_labels],[y,fake_labels])
        fmt = "%s [adversarial loss: %f, srcloss: %f,lblloss: %f,srcacc: %f,lblacc: %f]"
        log = fmt % (log,metrics[0],metrics[1],metrics[2],metrics[3],metrics[4])
        print(log)
        
        if (i+1) % save_interval == 0:
            plot_images(generator,noise_input,noise_label,model_name = model_name)
    generator.save(model_name + ".h5")

## Building a PLot Image Function

In [ ]:
def plot_images(generator,noise_input,noise_label,show=False,step=0,model_name="gan"):
    os.makedirs(model_name, exist_ok=True)
    filename = os.path.join(model_name, "%05d.png" % step)
    images = generator.predict([noise_input,noise_label])
    plt.figure(figsize=(2, 2))
    num_images = images.shape[0]
    image_size = images.shape[1]
    rows = int(math.sqrt(noise_input.shape[0]))
    for i in range(num_images):
        plt.subplot(rows, rows, i + 1)
        image = np.reshape(images[i], [image_size, image_size])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        
    plt.savefig(filename)
    plt.show()

# Building  Adversarial Model

In [ ]:
def build_and_train_models():
    (x_train, y_train), (_, _) = mnist.load_data()
    image_size = x_train.shape[1]
    x_train = np.reshape(x_train, [-1, image_size, image_size, 1])
    x_train = x_train.astype('float32') / 255
    num_labels = len(np.unique(y_train))
    y_train = to_categorical(y_train)
    model_name = "acgan_mnist"
    latent_size = 100
    batch_size = 64
    train_steps = 40000
    lr = 2e-4
    decay = 6e-8
    input_shape = (image_size, image_size, 1)
    label_shape = (num_labels, )
    inputs = Input(shape=input_shape,name='discriminator_input')
    discriminator = Discriminator(inputs, num_labels=num_labels)
    optimizer = RMSprop(lr=lr, decay=decay)
    loss = ['binary_crossentropy', 'categorical_crossentropy']
    discriminator.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])
    discriminator.summary()

    
    input_shape = (latent_size, )
    inputs = Input(shape=input_shape, name='z_input')
    labels = Input(shape=label_shape, name='labels')

    generator = Generator(inputs,image_size,labels=labels)
    generator.summary()
    optimizer = RMSprop(lr=lr*0.5, decay=decay*0.5)
    discriminator.trainable = False
    adversarial = Model([inputs, labels],discriminator(generator([inputs, labels])),name=model_name)
    adversarial.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])
    adversarial.summary()
    models = (generator, discriminator, adversarial)
    data = (x_train, y_train)
    params = (batch_size, latent_size,train_steps, num_labels, model_name)
    train(models, data, params)

In [ ]:
build_and_train_models()